In [5]:
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
#metric = "f1 score - model labels"
metric = "sufficiency"
type_of_rat = "fixed"

In [ ]:
def export_legend(legend, filename="legend.png"):
    fig  = legend.figure
    fig.canvas.draw()
    bbox  = legend.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    fig.savefig(filename, dpi="figure", bbox_inches=bbox)

    return

datasets : list = ["sst", "multirc", "agnews", "evinf"]
metrics_folder : str = "faithfulness_metrics"
rationale_type: str = "topk"
metric : str = "sufficiency"
var_or_fixed : str = "fixed"
plot_legend : bool = False
y_label_at_metric : str = "f1 score - model labels")


assert metric in {"comprehensiveness", "sufficiency", "f1 score - model labels", "f1 score - actual labels"}, (
    """
    Must be one of the following metrics:
    * comprehensiveness
    * sufficiency
    * f1 score - model labels
    * f1 score - actual labels
    """
)

collective = {}

for task_name in datasets:

    fname = os.path.join(
        metrics_folder,
        task_name,
        f"{rationale_type}-test-increasing-feature-scoring.json"
    )


    with open(fname, "r") as file : data = json.load(file) 

    if "f1" not in metric:

        collective[task_name] = [x[metric]["mean"] for x in data[var_or_fixed].values()]

    else:

        collective[task_name] = [x[metric] for x in data[var_or_fixed].values()]


df = pd.DataFrame(collective)

mapper = {
    "gradients" : "x∇x",
    "ig" : "IG",
    "deeplift" : "DeepLift",
    "attention" : "α",
    "scaled attention" : "α∇α",
    "lime" : "LIME"
}

nicer_tasknames = {
    "sst" : "SST",
    "evinf" : "Ev.Inf.",
    "multirc" : "MultiRC",
    "agnews" : "AG"
}



df.index = ["\n".join([mapper[y] for y in x.split("-")]) for x in data[var_or_fixed].keys()]
df = df.rename(columns = nicer_tasknames)

df = df.iloc[::-1]

plt.style.use('tableau-colorblind10')
# plt.style.use("seaborn-colorblind")
plt.rcParams['axes.edgecolor']='#333F4B'
plt.rcParams['axes.linewidth']=0.2
plt.rcParams['xtick.color']='#333F4B'
plt.rcParams['ytick.color']='#333F4B'
plt.rcParams["font.variant"] = "small-caps"

fig, ax = plt.subplots(figsize = (8,10))

plt.yticks(fontsize = 35, rotation = 45)

#     ax.set_xlabel(metric.capitalize(), fontsize = 35)

df.T.plot.barh(ax = ax, legend=False, width=0.75)


if metric != y_label_at_metric:

    y_axis = ax.axes.get_yaxis()
    y_axis.set_visible(False)
    ax.set_yticklabels([])


bars = ax.patches
hatches = ''.join(h*len(df.T) for h in 'x/O+*.')

for bar, hatch in zip(bars, hatches):
    bar.set_hatch(hatch)

if "f1" not in metric:

    ax.set_xticks(np.arange(0.,df.max().max(), 0.2))
    plt.xticks(fontsize=23)
    ax.set_xticks(np.arange(0.,df.max().max()+0.025, 0.025), minor=True)


    if plot_legend:
        handles, labels = ax.get_legend_handles_labels()
        legend = plt.legend( 
            handles[::-1], labels[::-1],
            loc = (1,0), 
            fontsize = 20, 
            fancybox = True, 
            framealpha = 0.4,
            labelspacing = 1,
            ncol=2,
            columnspacing=0.1,
            frameon=False
        )

else:

    ax.set_xticks(np.arange(0,100+20,20))
    plt.xticks(fontsize=23)
    ax.set_xticks(np.arange(0,100,5), minor=True)

    if plot_legend:
        handles, labels = ax.get_legend_handles_labels()
        legend = plt.legend( 
            handles[::-1], labels[::-1],
            loc = (1,0), 
            fontsize = 20, 
            fancybox = True, 
            framealpha = 0.4,
            ncol=2,
            frameon=False
        )

plt.grid(which='minor', alpha=0.15)
plt.grid(which='major', alpha=0.5)
plt.tight_layout()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

if plot_legend:

    export_legend(legend, filename = "graphs_and_tables/increasing-set-of-feature_scoring/legend.png")

plt.show()
#     os.makedirs("graphs_and_tables/increasing-set-of-feature_scoring/", exist_ok = True)

#     fig.savefig(f"graphs_and_tables/increasing-set-of-feature_scoring/{metric}-increasing-feat-attr.png", dpi = 300)#, bbox_inches="tight")

plt.close()

In [7]:
datasets : list = ["sst"]
metrics_folder : str = "faithfulness_metrics"
rationale_type: str = "topk"
metric : str = "sufficiency"
var_or_fixed : str = "fixed"
plot_legend : bool = False
y_label_at_metric : str = "f1 score - model labels"


assert metric in {"comprehensiveness", "sufficiency", "f1 score - model labels", "f1 score - actual labels"}, (
    """
    Must be one of the following metrics:
    * comprehensiveness
    * sufficiency
    * f1 score - model labels
    * f1 score - actual labels
    """
)

collective = {}

for task_name in datasets:

    fname = os.path.join(
        metrics_folder,
        task_name,
        f"{rationale_type}-test-faithfulness-metrics-description.json"
    )


    with open(fname, "r") as file : data = json.load(file) 

    if "f1" not in metric:

        collective[task_name] = [x[metric]["mean"] for x in data[var_or_fixed].values()]

    else:

        collective[task_name] = [x[f"{var_or_fixed}-{metric}"] for x in data.values()]


KeyError: 'fixed'

In [21]:
new_data_means = {}
new_data_stds = {}

for task_name in ["sst", "agnews", "evinf", "multirc"]:
    
    fname = os.path.join(
        metrics_folder,
        task_name,
        f"{rationale_type}-test-faithfulness-metrics-description.json"
    )


    with open(fname, "r") as file : data = json.load(file) 
    
    new_data_means[task_name] = {}
    new_data_stds[task_name] = {}
    
    for feat_attr in ["deeplift", "lime", "attention", "scaled attention", "ig", "gradients", "fixed-len_var-feat"]:
        
              new_data_means[task_name][feat_attr] =  data[f"{var_or_fixed}-{feat_attr}"]["comprehensiveness"]["mean"]
        
#             new_data_means[task_name][feat_attr] = {
#                 "comprehensiveness" : data[f"{var_or_fixed}-{feat_attr}"]["comprehensiveness"]["mean"],
#                 "sufficiency" : data[f"{var_or_fixed}-{feat_attr}"]["sufficiency"]["mean"]
#             }
            
#             new_data_stds[task_name][feat_attr] = {
#                 "comprehensiveness" : data[f"{var_or_fixed}-{feat_attr}"]["comprehensiveness"]["std"],
#                 "sufficiency" : data[f"{var_or_fixed}-{feat_attr}"]["sufficiency"]["std"]
#             }

In [22]:
pd.DataFrame(new_data_means)

sst    agnews     evinf   multirc
deeplift            0.331795  0.207409  0.212552  0.370821
lime                0.419343  0.245370  0.255738  0.421006
attention           0.344912  0.260587  0.563640  0.297386
scaled attention    0.542094  0.284917  0.796146  0.337897
ig                  0.400623  0.238320  0.354385  0.383036
gradients           0.246823  0.159030  0.264583  0.391001
fixed-len_var-feat  0.568554  0.408156  0.838951  0.593546